In [2]:
hub_token = "<HUB_TOKEN>"

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict

dataframe = pd.concat(
    [
        pd.read_csv("./data/hateval2019_en_train.csv"),
        pd.read_csv("./data/hateval2019_en_dev.csv"),
        pd.read_csv("./data/hateval2019_en_test.csv"),
    ],
    keys=["train", "dev", "test"],
    names=["split", "index"],
)

datasets = DatasetDict(
    {
        split: Dataset.from_pandas(dataframe.loc[(split)])
        for split in ["train", "dev", "test"]
    }
)
datasets


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'HS', 'TR', 'AG', 'index'],
        num_rows: 9000
    })
    dev: Dataset({
        features: ['id', 'text', 'HS', 'TR', 'AG', 'index'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'text', 'HS', 'TR', 'AG', 'index'],
        num_rows: 3000
    })
})

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained(
    "vinai/bertweet-covid19-base-cased", normalization=True
)
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/bertweet-covid19-base-cased"
)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/vinai/bertweet-covid19-base-cased/resolve/main/config.json from cache at /home/chris-zeng/.cache/huggingface/transformers/b0d7660a1cf1cc386b57bf9307b4bc6f23b17e384049be92e2068b42dd6faafc.a2b6026e688d1b19cebc0981d8f3a5b1668eabfda55b2c42049d5eac0bc8cb2d
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-covid19-base-cased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_ty

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_preds):
    pred_logits, labels_logits = eval_preds
    preds = pred_logits.argmax(axis=1)
    labels = labels_logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro"),
    }


In [6]:
import numpy as np


def indice2logits(indice, num_classes):
    indice = np.array(indice)
    logits = np.zeros([len(indice), num_classes], dtype=float)
    logits[np.arange(len(indice)), indice] = 1.0
    return {"label_logits": logits}


datasets = datasets.map(
    lambda rec: tokenizer(
        rec["text"],
        padding="max_length",
        max_length=192,
        pad_to_multiple_of=8,
        return_token_type_ids=True,
        return_attention_mask=True,
    ),
    batched=True,
    keep_in_memory=True,
    batch_size=128,
)

datasets = datasets.map(
    lambda rec: indice2logits(rec["HS"], 2),
    batched=True,
    keep_in_memory=True,
)

datasets = datasets.rename_column("label_logits", "labels")
datasets = datasets.remove_columns([])
datasets


  0%|          | 0/71 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'HS', 'TR', 'AG', 'index', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9000
    })
    dev: Dataset({
        features: ['id', 'text', 'HS', 'TR', 'AG', 'index', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'text', 'HS', 'TR', 'AG', 'index', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="outputs/bertweet",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=30,
    logging_strategy="epoch",
    remove_unused_columns=True,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=128,
    optim="adamw_apex_fused",
    bf16=True,
    tf32=True,
    learning_rate=1e-6,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=True,
    hub_strategy="all_checkpoints",
    hub_model_id="ChrisZeng/bertweet-base-cased-covid19-hateval",
    hub_token=hub_token,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["dev"],
    compute_metrics=compute_metrics,
)

trainer_output = trainer.train(
    resume_from_checkpoint=True,
)
trainer.save_model()


In [9]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained(
    "ChrisZeng/bertweet-base-cased-covid19-hateval"
)
model = AutoModelForSequenceClassification.from_pretrained(
    "ChrisZeng/bertweet-base-cased-covid19-hateval"
)

training_args = TrainingArguments(
    output_dir="outputs/inference",
    overwrite_output_dir=True,
    remove_unused_columns=True,
    eval_accumulation_steps=128,
    disable_tqdm=True,
)

trainer = Trainer(model=model, args=training_args)


preds_test = trainer.predict(datasets["test"]).predictions.argmax(axis=1)
preds_dev = trainer.predict(datasets["dev"]).predictions.argmax(axis=1)

pd.DataFrame(
    {
        "dev": {
            "accuracy": accuracy_score(datasets["dev"]["HS"], preds_dev),
            "f1": f1_score(datasets["dev"]["HS"], preds_dev, average="macro"),
        },
        "test": {
            "accuracy": accuracy_score(datasets["test"]["HS"], preds_test),
            "f1": f1_score(datasets["test"]["HS"], preds_test, average="macro"),
        },
    }
).transpose()


loading file https://huggingface.co/ChrisZeng/bertweet-base-cased-covid19-hateval/resolve/main/vocab.txt from cache at /home/chris-zeng/.cache/huggingface/transformers/a65a5273741a95af789ee6a68080517bc0814ee866519c1abf0635f8699f3826.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982
loading file https://huggingface.co/ChrisZeng/bertweet-base-cased-covid19-hateval/resolve/main/bpe.codes from cache at /home/chris-zeng/.cache/huggingface/transformers/e27b892496ae2e66b946e87a21ef55b71bc2823ca0bc3d534ef5333f06f20f1e.75877d86011e5d5d46614d3a21757b705e9d20ed45a019805d25159b4837b0a4
loading file https://huggingface.co/ChrisZeng/bertweet-base-cased-covid19-hateval/resolve/main/added_tokens.json from cache at /home/chris-zeng/.cache/huggingface/transformers/7244bc040ffc27b11cbd505436c8c8fb6d30f66c64cb7e608bdea8e25283f249.c1e7052e39d2135302ec27455f6db22e1520e6539942ff60a849c7f83f8ec6dc
loading file https://huggingface.co/ChrisZeng/bertweet-base-cased-covid19-hateval/resolve/main/spe

,accuracy,f1
dev,0.772000,0.771176
test,0.553667,0.524164
